# Recap 12 & Examples
## Module structure
The `local_optimisation` module exposes functions `bfgs`, `gradient_descent`, `hybrid_eigenvector_following`, and `line_search`.

The `matrices` module exposes functions `solve_system`, `determinant`, `matrix_inverse`, and `matrix_pseudoinverse`.

The `pair_potential` module exposes functions `lennard_jones_potential`, `pair_potential`, `lj_energy_c`, and `lj_gradient_c`.

## Gradient of a pair potential

The Lennard-Jones pair energy and gradient $(\epsilon = \sigma = 1)$:
\begin{align}
\upsilon_{\text{LJ}}{(r)} &= 4\left[ \left(\frac{1}{r}\right)^{12} - \left(\frac{1}{r}\right)^{6}\right] \\
\frac{\text{d}}{\text{d}r}\upsilon_{\text{LJ}}{(r)} &= \frac{24}{r}\left[ \left(\frac{1}{r}\right)^{6} - 2\left(\frac{1}{r}\right)^{12}\right] 
\end{align}

The total pairwise energy and gradient:
\begin{align}
V{(\mathbf{r})} &=\sum_{i\ne j} \upsilon{(r_{ij})} \\
\frac{\partial}{\partial r_{i\alpha}} V{(\mathbf{r})} &= \sum_{i\ne j} \frac{\partial}{\partial r_{i\alpha}} \upsilon{(r_{ij})} \\
&= \sum_{i\ne j} \frac{\partial r_{ij}}{\partial r_{i\alpha}} \frac{\partial}{\partial r_{ij}}\upsilon{(r_{ij})} \\
&= \sum_{i\ne j} \frac{r_{i\alpha}-r_{j\alpha}}{r_{ij}} \frac{\text{d}}{\text{d}r_{ij}}\upsilon{(r_{ij})} \\
\text{where} \quad r_{ij} &= \left[\left(r_{ix}-r_{jx}\right)^{2}+\left(r_{iy}-r_{jy}\right)^{2}+\left(r_{iz}-r_{jz}\right)^{2}\right]^{\frac{1}{2}} \\
\frac{\partial r_{ij}}{\partial r_{ix}} &= 2\left(r_{ix}-r_{jx}\right) \cdot \frac{1}{2}\left[\left(r_{ix}-r_{jx}\right)^{2}+\left(r_{iy}-r_{jy}\right)^{2}+\left(r_{iz}-r_{jz}\right)^{2}\right]^{-\frac{1}{2}} \\
&= \frac{r_{ix}-r_{jx}}{\left[\left(r_{ix}-r_{jx}\right)^{2}+\left(r_{iy}-r_{jy}\right)^{2}+\left(r_{iz}-r_{jz}\right)^{2}\right]^{\frac{1}{2}}} \\
&= \frac{r_{ix}-r_{jx}}{r_{ij}}
\end{align}


## `cython` implementation of the Lennard-Jones potential
Python is easy to learn but slow as sin. This downside can be mitigated by using `cython`, a module that allows use to write python code that can be compiled as c code. Although plain Python can be compiled in this way, taking full advantage of cython requires certain extra directives.

In the `pair_potential` directory you will find a pure python implementation of the generic pair potential and LJ potential and gradient, as well as a cython implementations of the pair potential specificially for the LJ potential (`lj_potential_c.pyx`).

### Compiling
If you want to use the functions `lj_energy_c` and `lj_gradient_c`, you must first compile the code.

1. download `examples.zip` from the SEP and extract its contents
1. create a PyCharm project in the top directory, `examples`
1. create a new Python environment, and install the dependencies list in `requirements.txt`
1. open a terminal session
1. navigate to the top directory, `examples`
1. run `python setup.py build --in-place`

If the next two cells run without errors, you have successful compiled the code and can use it for the examples class (and your final project).

In [5]:
import numpy as np
from pair_potential import lj_energy_c, lj_gradient_c
from local_optimisation import gradient_descent, bfgs, hybrid_eigenvector_following

In [6]:
np.random.seed(0)
pos = np.load("config.npy")

f = lj_energy_c
df = lj_gradient_c

print(lj_energy_c(pos), np.linalg.norm(lj_gradient_c(pos)))

minimum = bfgs(f, df, pos, xtol=1e-8, gtol=1e-5)
print(lj_energy_c(minimum), np.linalg.norm(lj_gradient_c(minimum)))

saddle = hybrid_eigenvector_following(f, df, pos, tolg=1e-6, tolev=1e-6)
print(lj_energy_c(saddle), np.linalg.norm(lj_gradient_c(saddle)))

-37.03053758273098 4.169237684330122
-39.75218251560557 1.906933159170371e-06
-37.18288685363649 3.221596241579343e-07
